In [177]:
chapter = "chapter3/"
img_path = "img/content/"+chapter

In [178]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import rv_continuous
from sklearn.gaussian_process.kernels import Matern
from KKKF.DynamicalSystems import DynamicalSystem
from KKKF.kEDMD import KoopmanOperator

# Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import kaleido

import os

In [179]:
root = "/Users/diegoolguin/Koopman_nonlinear_filter"
if not os.getcwd()==root:
    os.chdir(root)

In [180]:
# Matriz del modelo lineal
alpha = -0.5
A = np.array([
    [0.01, 0.04, 0.0],
    [0.01, 0.02, alpha],
    [0.0, 0.04, 0.02]
])

# Dinámica del sistema
def f(x):
    return x + A@x

# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 2000
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

class product_dist(rv_continuous):
    def _rvs(self, size=1, random_state=stats.randint(1,10000).rvs()):
        return np.array([stats.uniform(-1,1).rvs(size=size, random_state=random_state),
                         stats.uniform(-1,1).rvs(size=size, random_state=random_state),
                         stats.uniform(-1,1).rvs(size=size, random_state=random_state)])

# Distribuciones

X_dist = product_dist()
X_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=5*np.eye(3))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(3))
obs_dist = stats.multivariate_normal(mean=np.zeros(1), cov=1e-7*np.eye(1))

# Sistema dinámico
dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

# Datos sintéticos
iters = 50
x0 = np.array([0.1, 0.1, 0.1])
x = np.zeros((iters, nx))

x[0] = x0

for i in range(1, iters):
    x[i] = f(x[i-1])

# Operador de Koopman
Koop = KoopmanOperator(k, dyn)
Koop.compute_edmd(N)

U, B, phi = Koop.U, Koop.B, Koop.phi

# Sistema con Koopman
x_koop = np.zeros((iters, nx))
x_koop[0] = B@phi(x0)
z = np.zeros((iters, N))
z[0] = phi(x0)

for i in range(1, iters):
    z[i] = U@z[i-1]
    x_koop[i] = B@z[i]

t = np.arange(iters)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos",
    xaxis_title="Tiempo",
    yaxis_title="Estados"
    )

fig.write_image(img_path+"Linear1.pdf")
fig.show()

In [181]:
# Dinámica del sistema
def f(x):
    return x + A@x

# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 2500
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=10*np.eye(3))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(3))
obs_dist = stats.multivariate_normal(mean=np.zeros(1), cov=1e-7*np.eye(1))

# Datos sintéticos
iters = 50
x0 = np.array([0.1, 0.1, 0.1])

# Tiempo
t = np.arange(iters)

In [182]:
def compute_linear_kedmd(alpha, N):
    # Matriz
    A = np.array([
        [0.01, 0.04, 0.0],
        [0.01, 0.02, alpha],
        [0.0, 0.04, 0.02]
    ])

    # Función de dinámica
    f = lambda x: x + A@x

    # Solución real
    x = np.zeros((iters, nx))
    x[0] = x0

    for i in range(1, iters):
        x[i] = f(x[i-1])

    # Sistema dinámico
    dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

    # Operador de Koopman
    Koop = KoopmanOperator(k, dyn)

    # Se calcula kEDMD
    Koop.compute_edmd(N)
    U, B, phi = Koop.U, Koop.B, Koop.phi

    # Sistema con Koopman
    x_koop = np.zeros((iters, nx))
    x_koop[0] = B@phi(x0)
    z = np.zeros((iters, N))
    z[0] = phi(x0)

    for i in range(1, iters):
        z[i] = U@z[i-1]
        x_koop[i] = B@z[i]

    return x, x_koop

In [183]:
# Alpha
alpha = -0.3

# Predicción
x, x_koop = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos",
    xaxis_title="Tiempo",
    yaxis_title="Estados"
    )

fig.write_image(img_path+"Linear1.pdf")
fig.show()

In [184]:
# Alpha
alpha = -0.1

# Predicción
x, x_koop = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos, matriz de parámetro {alpha}",
    xaxis_title="Tiempo",
    yaxis_title="Estados"
    )

fig.write_image(img_path+"Linear2.pdf")
fig.show()

In [185]:
# Alpha
alpha = 0.05

# Predicción
x, x_koop = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos, matriz de parámetro {alpha}",
    xaxis_title="Tiempo",
    yaxis_title="Estados"
    )

fig.write_image(img_path+"Linear3.pdf")
fig.show()

In [ ]:
alpha = 0.05
Ns = np.arange(100, 2100, 100)

errors = np.zeros_like(Ns)

for i in range(len(Ns)):
    x, x_koop = compute_linear_kedmd(alpha, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Plot


In [186]:
# Parámetros del modelo SIR
beta, gamma = 1, 0.3

# Dinámica del sistema
def f(x):
    return np.array([x[0]-beta*x[0]*x[1] ,x[1]+beta*x[0]*x[1] - gamma*x[1], x[2]+gamma*x[1]])

# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 500
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.dirichlet(alpha=1/3*np.ones(nx))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(3))
obs_dist = stats.multivariate_normal(mean=np.zeros(1), cov=1e-7*np.eye(1))

# Sistema dinámico
dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

# Datos sintéticos
iters = 20
x0 = np.array([0.9, 0.1, 0.0])
x = np.zeros((iters, nx))

x[0] = x0

for i in range(1, iters):
    x[i] = f(x[i-1])

# Operador de Koopman
Koop = KoopmanOperator(k, dyn)
Koop.compute_edmd(N)

U, B, phi = Koop.U, Koop.B, Koop.phi

# Sistema con Koopman
x_koop = np.zeros((iters, nx))
x_koop[0] = B@phi(x0)
z = np.zeros((iters, N))
z[0] = phi(x0)

for i in range(1, iters):
    z[i] = U@z[i-1]
    x_koop[i] = B@z[i]

t = np.arange(iters)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo SIR de parámetros ({beta}, {gamma}) con N = {N} puntos",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada"
    )

fig.write_image(img_path+"SIR.pdf")
fig.show()

In [187]:
f"kEDMD para modelo SIR con N = {N} puntos y"+r" $\beta={}$".format(beta)

'kEDMD para modelo SIR con N = 500 puntos y $\\beta=1$'